In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [41]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import KFold, GridSearchCV

In [42]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')
df

In [43]:
df.isna().sum().sum()

In [44]:
df.info()

In [45]:
df["target"].value_counts()

In [46]:
df.describe()

In [47]:
df["f_27"].nunique()

In [48]:
df["f_27"].value_counts()

In [49]:
df["f_27"].describe()

In [50]:
df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

In [51]:
X = df.drop(["id", "f_27", "target"], axis=1)

In [52]:
y = df.target.values

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [59]:
params = {
        #'min_child_weight':[5, 10],
        #'gamma':[2, 5],
        #'subsample':[0.8, 1.0],
        #'colsample_bytree':[0.8, 1.0],
        'max_depth':[5, 10],
        'n_estimators':[30, 120]
        }

clf = xgb.XGBClassifier(learning_rate=1, objective='binary:logistic',
                    silent=True, nthread=1)

kfold = KFold(n_splits=2)


In [60]:
grid = GridSearchCV(
  clf,
  param_grid=params,
  cv=kfold,
  scoring='roc_auc',
  refit=True,
  verbose=3,
  n_jobs=-1
)

In [61]:
model = grid.fit(X_train, y_train)

In [62]:
pred = model.predict(X_test)
print('Best AUC Score: {}'.format(model.best_score_))
print(confusion_matrix(y_test,pred))